<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

<h2>Herencia</h2>

El concepto de herencia en programación orientada a objetos nos permite aprovechar código de las clases de las cuales se hereda. La herencia nos permite representar la relación del tipo "el objeto B es un objeto A, pero con ciertas diferencias".

Una clase hija (o subclase) corresponde a una <b>especialización</b> de su clase padre. Cuando un objeto pertenece a una clase en particular, si esta clase es a su vez una subclase de otra clase más general, la herencia nos permite "heredar" los datos y comportamiento de la clase "madre" (superclase), de tal manera de no tener que volver a definir esos datos y comportamiento en la subclase. Por ejemplo: La clase "furgón escolar" es una subclase de la clase "vehículo", por lo tanto sabemos que la clase "furgón escolar" va a heredar los datos y comportamiento de "vehículo" (ruedas, motor, etc.) y no es necesario volver a definirlos en la subclase "furgón escolar". Lo interesante es que la subclase "furgón escolar" tiene ciertos datos y métodos que la hacen más especializada que la clase "vehículo", <i>i.e.</i>, lista de niños inscritos en el furgón.

También la herencia nos permite sobrescribir los métodos que necesitemos modificar. En Python, simplemente definimos nuevamente el método y con eso se entiende que la versión implementada en la subclase es la que cuenta. Una de las cosas que podemos hacer con herencia es extender los _built-ins_, por ejemplo, si queremos extender la clase "lista", podemos definir una subclase que heredará los métodos de la clase "lista" y a su vez tendrá datos y métodos propios:


In [2]:
# Aquí estamos extendiendo y especializando la clase lista estándar. Tiene todos los métodos de la lista más los definidos por
# nosotros. Recordar que para nombrar las clases se utiliza notación CamelCase.
class ContactList(list):
    
    # buscar es un método específico de esta sub-clase
    def buscar(self, nombre):
        matches = []
        
        for contacto in self:
            if nombre in contacto.nombre:
                matches.append(contacto)
                
        return matches

    
class Contacto:
    
    # Contacto se compone de una lista de contactos del tipo ContactList
    # contactos_list = [] #así sería para usar una lista común y corriente
    contactos_list = ContactList()

    def __init__(self, nombre, email):
        self.nombre = nombre
        self.email = email
        Contacto.contactos_list.append(self) # el método append() es heredado de la clase List()


# Familiar es una clase especializada de contacto que permite incluir el tipo de relación
class Familiar(Contacto):

    def __init__(self, nombre, email, relacion): # Overriding sobre el método __init__()
        super().__init__(nombre, email) # Obtiene la instancia del padre y llama a su funcion __init__
        self.relacion = relacion

In [3]:
p1 = Familiar(nombre = "Juan Gómez", email = "jg@hotmail.com", relacion = "padre")
p2 = Contacto(nombre = "Jorge González", email = "jg@gmail.com")
p3 = Familiar(nombre = "Pablo Gómez", email = "pab_g@gmail.com", relacion = 'primo')
p4 = Contacto(nombre = "Jorge Contreras", email = "jc@gmail.com")

L = [c.nombre for c in p1.contactos_list.buscar("Jorge")]

print('[', end='')
print(*L, sep=', ', end='')
print(']')

[Jorge González, Jorge Contreras]


<h2> Multiherencia</h2>

Tal como es posible que una subclase herede datos y comportamiento de una superclase, también es posible heredar
de más de una clase a la vez:


In [1]:
class Investigador:
    def __init__(self, area):
        self.area = area
        
class Docente:
    def __init__(self, Departamento):
        self.departamento = Departamento
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, area_investigacion, departamento):
        #esto no es del todo correcto, coming soon...
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre

p1 = Academico("Juan Perez", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Juan Perez
Inteligencia de Máquina
Ciencia De La Computación


<h2> Multiherencia y el Problema del diamante</h2>

El siguiente ejemplo muestra lo que ocurre en un contexto de multiherencia si es que cada sub-clase llama directamente a inicializar a todas sus superclases. La figura siguiente muestra la jerarquía de las clases en cuestión

![Diamante](img_diamante.png)

El siguiente código muestra qué ocurre cuando llamamos al método "llamar()" en ambas super clases desde la clase "SubClaseA".

In [1]:
class ClaseB:
    num_llamadas_B = 0
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    num_llamadas_izq = 0
    def llamar(self):
        ClaseB.llamar(self)
        print("Llamando método en Subclase izquierda")
        self.num_llamadas_izq += 1

class SubClaseDerecha(ClaseB):
    
    num_llamadas_der = 0
    
    def llamar(self):
        ClaseB.llamar(self)
        print("Llamando método en Subclase derecha")
        self.num_llamadas_der += 1

class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    num_llamadas_subA = 0
    def llamar(self):
        SubClaseIzquierda.llamar(self)
        SubClaseDerecha.llamar(self)
        print("Llamando método en SubclaseA")
        self.num_llamadas_subA += 1



s = SubClaseA()
s.llamar()
print(s.num_llamadas_subA, s.num_llamadas_izq, s.num_llamadas_der, s.num_llamadas_B)



Llamando método en Clase B
Llamando método en Subclase izquierda
Llamando método en Clase B
Llamando método en Subclase derecha
Llamando método en SubclaseA
1 1 1 2


Del output se puede apreciar que la clase de más arriba en la jerarquía ("Clase B"), fue llamada dos veces, a pesar de que
nuestra intención era llamarla sólo una vez. La estructura de jerarquía en forma de diamante ocurre siempre que tengamos una clase que hereda de dos clases, ya que como en Python todo es un objeto, todo hereda de la clase "object" (ver: "new style classes (https://www.python.org/doc/newstyle/)"), por lo tanto en general el esquema de multiherencia se ve de la siguiente forma:

![Diamante2](img_diamante_2.png)
Siguiendo el mismo ejemplo anterior, en vez de llamar al método "llamar()", llamamos al método "__init__", estaríamos inicializando dos veces en la clase "object"!!. 


<h2>Solución:</h2>

La solución es que cada clase debe preocuparse de llamar a inicializar a la clase que la "precede" en el orden del esquema de la multiherencia. En Python el orden de las clases va de izquierda a derecha dentro de la lista de super-clases desde donde hereda la sub-clase. En este caso, simplemente debemos preocuparnos de hacer una llamada a "super()", Python se encargará de que la llamada corresponda a la clase que respeta el orden en la multiherencia, en este caso, después de la subclase viene la clase "SubclaseIzquierda", después "SubClaseDerecha" y finalmente "ClaseB"

In [2]:
class ClaseB:
    num_llamadas_B = 0
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    num_llamadas_izq = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en Subclase Izquierda")
        self.num_llamadas_izq += 1

class SubClaseDerecha(ClaseB):
    num_llamadas_der = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en Subclase Derecha")
        self.num_llamadas_der += 1

class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    num_llamadas_subA = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en SubclaseA")
        self.num_llamadas_subA += 1


s = SubClaseA()
s.llamar()
print(s.num_llamadas_subA, s.num_llamadas_izq, s.num_llamadas_der, s.num_llamadas_B)



Llamando método en Clase B
Llamando método en Subclase Derecha
Llamando método en Subclase Izquierda
Llamando método en SubclaseA
1 1 1 1


### El método `__mro__` (method resolution order) nos muestra el orden de la jerarquía. 

Es útil para casos de multiherencia más complejos, Python utiliza el algoritmo C3 para calcular un orden lineal entre las clases que participan en el esquema de multiherencia: 


In [30]:
SubClaseA.__mro__

(__main__.SubClaseA,
 __main__.SubClaseIzquierda,
 __main__.SubClaseDerecha,
 __main__.ClaseB,
 object)

Este es un ejemplo de estructura de multiherencia que no estaría permitida en Python, ya que el algoritmo C3 generaría un error:

In [31]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(Y, X):
     def call_me(self):
         print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

In [32]:
print(B.__mro__)

(<class '__main__.B'>, <class '__main__.Y'>, <class '__main__.X'>, <class 'object'>)


<h2> Ejemplo Multiherencia: </h2>

<h3> Mala práctica: </h3> 

Si bien el siguiente ejemplo al parecer funciona correctamente, al llamar directamente a las superclases dentro de la clase Cliente estamos cometiendo el error mencionado anteriormente (se está llamando al inicializador de "object" más de una vez)

In [11]:
class AddressHolder:
    def __init__(self, calle, numero, comuna, ciudad):
        self.calle = calle
        self.ciudad = ciudad
        self.comuna = comuna
        self.numero = numero

class Contacto:

    contactos_list = []

    def __init__(self, nombre, email):
        self.nombre = nombre
        self.email = email
        Contacto.contactos_list.append(self)


class Cliente(Contacto, AddressHolder):

    def __init__(self, nombre, email, telefono, calle, numero, comuna, ciudad):
        Contacto.__init__(self, nombre, email)
        AddressHolder.__init__(self, calle, numero, comuna, ciudad)
        self.telefono = telefono

c = Cliente('Juan Perez', 'jp@gmail.com', '23542331', 'Pedro de Valdivia', '231', 'Providencia', 'Santiago')

print("{}, {}, {}, {}".format(c.nombre, c.email, c.calle, c.comuna))

Juan Perez, jp@gmail.com, Pedro de Valdivia, Providencia


### Forma mejorada: `*args` y `**kwargs`

Antes de ver una forma más adecuada para la versión del código anterior, mostraremos cómo usar "\*\*kwargs". En este caso \*\*kwargs se refiere a un "keyworded variable-length argument list", donde \*\* mapea los elementos contenidos en el diccionario <i>kwargs</i> y los pasa a la función como argumentos no posicionales. Este método puede ser usado para enviar una cantidad variable de argumentos a una función:

In [17]:
def metodo(arg1, arg2, arg3):
    print("arg1: {}".format(arg1))
    print("arg2: {}".format(arg2))
    print("arg3: {}".format(arg3))

kwargs = {"arg3": 3, "arg2": "two"}
metodo(1, **kwargs)

arg1: 1
arg2: two
arg3: 3


De forma casi análoga, \*args se refiere a un "Non-keyworded variable length argument list", donde el operador `*` desempaqueta el contenido del iterable `args` y los pasa a la función como argumentos posicionales. La principal diferencia es que la lista de argumentos contiene simplemente los valores, sin los keywords (guardada en una lista):

In [22]:
def metodo2(f_arg, *argv):
    print("primer arg normal: {}".format(f_arg))
    for arg in argv:
        print("siguiente argumento de *argv : {}".format(arg))

metodo2('hola','como','va','todo')

primer arg normal: hola
siguiente argumento de *argv : como
siguiente argumento de *argv : va
siguiente argumento de *argv : todo


Otro ejemplo:

In [17]:
def funcion(a=0, b=0):
    return a+b

# Usando solo un valor posicional y el resto usa los argumentos por defecto
valores = (1,)
print(funcion(*valores))

# Usando todos los argumentos posicionales definidos en la lista
valores = (1,2)
print(funcion(*valores))

# La función necesita dos argumentos, por lo tanto el exceso de argumentos posicionales genera un error.
valores = (1,2,3)
print(funcion(*valores))

1
3


TypeError: funcion() takes from 0 to 2 positional arguments but 3 were given

Cuando usamos en una llamada `*args` y `**kwargs` juntos, se debe usar el siguiente orden: `alguna_funcion(f_args,*args,**kwargs)`

In [4]:
class AddressHolder:
    def __init__(self, calle='', ciudad='', numero='', comuna='',**kwargs):
        print(kwargs, "_____")
        super().__init__(**kwargs)
        self.calle = calle
        self.ciudad = ciudad
        self.comuna = comuna
        self.numero = numero


class Contacto:

    contactos_list = []

    def __init__(self, nombre = '', email = '', **kwargs):
        print(kwargs, "_____")
        super().__init__(**kwargs)
        self.nombre = nombre
        self.email = email
        Contacto.contactos_list.append(self)


class Cliente(Contacto, AddressHolder):#Notar aquí que la clase Contacto fue involucrada (a posterior)
                                       #en un contexto de multiherencia
    def __init__(self, telefono='', **kwargs):
        print(kwargs, "_____")
        super().__init__(**kwargs)
        self.telefono = telefono

print(Cliente.__mro__)

(<class '__main__.Cliente'>, <class '__main__.Contacto'>, <class '__main__.AddressHolder'>, <class 'object'>)


In [5]:
c = Cliente(nombre = 'Juan Perez', email = 'jp@gmail.com', telefono = '23542331',
            calle = 'Pedro de Valdivia', numero = '231', comuna = 'Providencia', ciudad = 'Santiago')

print("{}, {}, {}, {}".format(c.nombre, c.email, c.calle, c.comuna))

{'nombre': 'Juan Perez', 'email': 'jp@gmail.com', 'calle': 'Pedro de Valdivia', 'numero': '231', 'comuna': 'Providencia', 'ciudad': 'Santiago'} _____
{'calle': 'Pedro de Valdivia', 'numero': '231', 'comuna': 'Providencia', 'ciudad': 'Santiago'} _____
{} _____
Juan Perez, jp@gmail.com, Pedro de Valdivia, Providencia
